In [6]:
# -*- coding: UTF-8 -*-

"""
训练 DCGAN
"""

import os
import glob#读取文件
import numpy as np
from scipy import misc#科学计算库
import tensorflow as tf

from network import *


def train():
    # 确保包含所有图片的 images 文件夹在所有 Python 文件的同级目录下
    # 当然了，你也可以自定义文件夹名和路径
    if not os.path.exists("images"):
        raise Exception("包含所有图片的 images 文件夹不在此目录下，请添加")

    # 获取训练数据
    data = []
    for image in glob.glob("images/*"):
        image_data = misc.imread(image)  # imread 利用 PIL 来读取图片数据
        data.append(image_data)
    input_data = np.array(data)

    # 将数据标准化成 [-1, 1] 的取值, 这也是 Tanh 激活函数的输出范围
    input_data = (input_data.astype(np.float32) - 127.5) / 127.5

    # 构造 生成器 和 判别器
    g = generator_model()
    d = discriminator_model()

    # 构建 生成器 和 判别器 组成的网络模型
    d_on_g = generator_containing_discriminator(g, d)

    # 优化器用 Adam Optimizer
    g_optimizer = tf.keras.optimizers.Adam(lr=LEARNING_RATE, beta_1=BETA_1)
    d_optimizer = tf.keras.optimizers.Adam(lr=LEARNING_RATE, beta_1=BETA_1)

    # 配置 生成器 和 判别器
    g.compile(loss="binary_crossentropy", optimizer=g_optimizer)
    d_on_g.compile(loss="binary_crossentropy", optimizer=g_optimizer)
    d.trainable = True
    d.compile(loss="binary_crossentropy", optimizer=d_optimizer)

    # 开始训练
    for epoch in range(EPOCHS):
        for index in range(int(input_data.shape[0] / BATCH_SIZE)):
            input_batch = input_data[index * BATCH_SIZE : (index + 1) * BATCH_SIZE]

            # 连续型均匀分布的随机数据（噪声）
            random_data = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))
            # 生成器 生成的图片数据
            generated_images = g.predict(random_data, verbose=0)

            input_batch = np.concatenate((input_batch, generated_images))#首尾相连，等于append，真实图像后跟生成图像
            output_batch = [1] * BATCH_SIZE + [0] * BATCH_SIZE

            # 训练 判别器，让它具备识别不合格生成图片的能力
            d_loss = d.train_on_batch(input_batch, output_batch)

            # 当训练 生成器 时，让 判别器 不可被训练
            d.trainable = False

            # 重新生成随机数据。很关键
            random_data = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))

            # 训练 生成器，并通过不可被训练的 判别器 去判别
            g_loss = d_on_g.train_on_batch(random_data, [1] * BATCH_SIZE)

            # 恢复 判别器 可被训练
            d.trainable = True

            # 打印损失
            print("Epoch {}, 第 {} 步, 生成器的损失: {:.3f}, 判别器的损失: {:.3f}".format(epoch, index, g_loss, d_loss))

        # 保存 生成器 和 判别器 的参数
        # 大家也可以设置保存时名称不同（比如后接 epoch 的数字），参数文件就不会被覆盖了
        if epoch % 10 == 9:
            g.save_weights("generator_weight", True)
            d.save_weights("discriminator_weight", True)


if __name__ == "__main__":
    train()


E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


Epoch 0, 第 0 步, 生成器的损失: 0.086, 判别器的损失: 0.674
Epoch 0, 第 1 步, 生成器的损失: 0.252, 判别器的损失: 0.656
Epoch 0, 第 2 步, 生成器的损失: 0.850, 判别器的损失: 0.539
Epoch 0, 第 3 步, 生成器的损失: 1.620, 判别器的损失: 0.446
Epoch 0, 第 4 步, 生成器的损失: 1.354, 判别器的损失: 0.529
Epoch 0, 第 5 步, 生成器的损失: 2.257, 判别器的损失: 0.365
Epoch 0, 第 6 步, 生成器的损失: 3.808, 判别器的损失: 0.342
Epoch 0, 第 7 步, 生成器的损失: 2.269, 判别器的损失: 0.539
Epoch 0, 第 8 步, 生成器的损失: 2.478, 判别器的损失: 0.438
Epoch 0, 第 9 步, 生成器的损失: 3.282, 判别器的损失: 0.414
Epoch 0, 第 10 步, 生成器的损失: 3.522, 判别器的损失: 0.482
Epoch 0, 第 11 步, 生成器的损失: 2.762, 判别器的损失: 0.528
Epoch 0, 第 12 步, 生成器的损失: 2.049, 判别器的损失: 0.552
Epoch 0, 第 13 步, 生成器的损失: 1.273, 判别器的损失: 0.715
Epoch 0, 第 14 步, 生成器的损失: 0.689, 判别器的损失: 0.750
Epoch 0, 第 15 步, 生成器的损失: 0.571, 判别器的损失: 0.599
Epoch 0, 第 16 步, 生成器的损失: 0.564, 判别器的损失: 0.526
Epoch 0, 第 17 步, 生成器的损失: 0.508, 判别器的损失: 0.508
Epoch 0, 第 18 步, 生成器的损失: 0.658, 判别器的损失: 0.564
Epoch 0, 第 19 步, 生成器的损失: 0.841, 判别器的损失: 0.410
Epoch 0, 第 20 步, 生成器的损失: 1.122, 判别器的损失: 0.401
Epoch 0, 第 21 步, 生成器的损失: 1.409, 判别器的损失: 0.39

KeyboardInterrupt: 